In [2]:
import requests
from requests_toolbelt.utils import dump

In [3]:
# простая функция, выводящая информациию о запросе, чтобы не копировать код много раз
# копипаста - это плохо
def print_request_info(r):
    print(r.status_code)
    print(r.text)    

## GET запросы

In [8]:
# простой GET запрос
r = requests.get("http://eu.httpbin.org/get")
print_request_info(r)


200
{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "eu.httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-635d5f48-16fe1744406f2d007420acf3"
  }, 
  "origin": "178.70.82.198", 
  "url": "http://eu.httpbin.org/get"
}



In [9]:
# Вот что происходит внутри
print(dump.dump_all(r).decode())

< GET /get HTTP/1.1
< Host: eu.httpbin.org
< User-Agent: python-requests/2.28.1
< Accept-Encoding: gzip, deflate
< Accept: */*
< Connection: keep-alive
< 

> HTTP/1.1 200 OK
> Date: Sat, 29 Oct 2022 17:13:44 GMT
> Content-Type: application/json
> Content-Length: 312
> Connection: keep-alive
> Server: gunicorn/19.9.0
> Access-Control-Allow-Origin: *
> Access-Control-Allow-Credentials: true
> 
{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "eu.httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-635d5f48-16fe1744406f2d007420acf3"
  }, 
  "origin": "178.70.82.198", 
  "url": "http://eu.httpbin.org/get"
}



То, что < - это мы шлем на сервер, то, что > - это ответ

Можете сразу обратить внимание на 2 хедера в ответе:  
&ensp;Access-Control-Allow-Origin: *  
&ensp;Access-Control-Allow-Credentials: true  
Мы про них еще вспомним

In [93]:
# раз мы увидели, что нужно слать, можем попробовать сами написать GET запрос, по честному, ручками
# \r\n - это специальные символы переноса строки

import select
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
# 1 параметр - тип сетевого протокола, в данном случае - IPv4. Например, можно поставить AF_INT6 - IPv6
# 2 параметр - тип протокола транспортного уровня, в данном случае TCP. например, можно ставить UDP
sock.connect(("eu.httpbin.org", 80))  # http:// здесь не пишется, это этажом выше, на уровне tcp никакого http не знают
msg = b"""
GET /get HTTP/1.1
Host: httpbin.org

"""
# в документации HTTP 1.1 написано, что обязательным является только хедер Host, так что не будем слать лишнего
sock.send(msg)

TIMEOUT = 1

while True:
    print(sock.recv(4096).decode())  # получаем 4096 байт, декодируем их (потому что они байты) и печатаем
    ready = select.select([sock], [], [], TIMEOUT)
    if not ready[0]:  # проверяем, что ещё есть, что получать
        break


HTTP/1.1 200 OK
Date: Fri, 21 Oct 2022 21:25:04 GMT
Content-Type: application/json
Content-Length: 198
Connection: keep-alive
Server: gunicorn/19.9.0
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true

{
  "args": {}, 
  "headers": {
    "Host": "httpbin.org", 
    "X-Amzn-Trace-Id": "Root=1-63530e30-1e65de483d4179282fa76cd4"
  }, 
  "origin": "178.70.82.198", 
  "url": "http://httpbin.org/get"
}



In [77]:
# GET запрос с заданными параметрами и хедерами
# Обратите внимание на строчку url в ответе
# Параметры при GET запросе кодируются прямо в url
print_request_info(
    requests.get("http://eu.httpbin.org/get", 
                 params={"test_param": "test", "t2": 1, "list": [1, 2, 3]}, 
                 headers={"My-Header": "test_value"})
)

200
{
  "args": {
    "list": [
      "1", 
      "2", 
      "3"
    ], 
    "t2": "1", 
    "test_param": "test"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "eu.httpbin.org", 
    "My-Header": "test_value", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-63509bfd-611d730011bdf9791ef8786a"
  }, 
  "origin": "178.70.82.198", 
  "url": "http://eu.httpbin.org/get?test_param=test&t2=1&list=1&list=2&list=3"
}



Числа, выводящиеся перед ответом - это код ответа, 200 - значит всё хорошо  
Но иногда бывает и плохо: https://ru.wikipedia.org/wiki/Список_кодов_состояния_HTTP  
Самые важные коды ошибок, про которые стоит прочитать: 400, 401, 403, 404, 405, 500  
Ещё есть 418

In [42]:
# что за число выводится после каждого запроса?
# это - код ответа, 200 - значит всё хорошо
# но иногда бывает и плохо: https://ru.wikipedia.org/wiki/Список_кодов_состояния_HTTP
# самые важные коды ошибок, про которые стоит прочитать: 400, 401, 403, 404, 405, 500
# ещё есть 418
code = 404
print_request_info(
    requests.get(f"http://eu.httpbin.org/status/{code}")
)

404



## POST запросы

Основное отличие в написании GET и POST запросов - передача параметров
В GET запросах, как было показано выше, парметры кодируются прямо в адресную строку.
В POST запросах параметры передаются в теле запроса

In [44]:
# простой POST запрос
# обратите внимание на появившееся поле data - раньше его не было
print_request_info(
    requests.post(f"http://eu.httpbin.org/post")
)

200
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "0", 
    "Host": "eu.httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-63508be2-515ea5ec7f2080810d03b95b"
  }, 
  "json": null, 
  "origin": "178.70.82.198", 
  "url": "http://eu.httpbin.org/post"
}



In [45]:
# тут в поле дата находится приведённый к строке json, а в поле json - распаршенный
# как видите, URL не изменился
# технически, можно и к POST запросу приклеить параметры, зашитые в адрес, но на практике так никто не делает
# почему - не знаю
print_request_info(
    requests.post(f"http://eu.httpbin.org/post", json={"test": "test", "param": [1, 2, 3]})
)


200
{
  "args": {}, 
  "data": "{\"test\": \"test\", \"param\": [1, 2, 3]}", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "36", 
    "Content-Type": "application/json", 
    "Host": "eu.httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-63508c23-273e3cd85e2f05df368ef499"
  }, 
  "json": {
    "param": [
      1, 
      2, 
      3
    ], 
    "test": "test"
  }, 
  "origin": "178.70.82.198", 
  "url": "http://eu.httpbin.org/post"
}



In [51]:
# библиотека requests любезно приводит переданный ей словарь к строке, но никто не запрещает нам сделать это самим
import json
print_request_info(
    requests.post(f"http://eu.httpbin.org/post", data=json.dumps({"test": "test"}))
)

200
{
  "args": {}, 
  "data": "{\"test\": \"test\"}", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "16", 
    "Host": "eu.httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-63509260-6dbd91b8451fce061ea54717"
  }, 
  "json": {
    "test": "test"
  }, 
  "origin": "178.70.82.198", 
  "url": "http://eu.httpbin.org/post"
}



можно заметить, что пропал хедер "Content-Type": "application/json", по-хорошему нам нужно его добавить
но можно и забить потому что: 

_Any HTTP/1.1 message containing an entity-body SHOULD include a Content-Type header field defining the media type of that body. If and only if the media type is not given by a Content-Type field, the recipient MAY attempt to guess the media type via inspection of its content and/or the name extension(s) of the URI used to identify the resource. If the media type remains unknown, the recipient SHOULD treat it as type "application/octet-stream".
(https://www.ietf.org/rfc/rfc2616.txt)_

Документация к requests наоборот, не советует ставить этот хедер вручную.
В общем, всем пофиг, работает - не трогай.

этот хедер может использоваться (а может и не использоваться) сервером, чтобы понять, как ему расшифровавать то, что пришло в теле запроса.
Как вы уже поняли, раз есть такой хедер, значит отправлять можно не только JSON'ы



In [48]:
# например можно отправить текст
print_request_info(
    requests.post(f"http://eu.httpbin.org/post", data="Hello")
)

200
{
  "args": {}, 
  "data": "Hello", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "5", 
    "Host": "eu.httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-63508dfb-13bf4d7564d4ce48100bc283"
  }, 
  "json": null, 
  "origin": "178.70.82.198", 
  "url": "http://eu.httpbin.org/post"
}



In [50]:
# или файл
# rb - read binary, отсылать файл нужно в бинарном формамате - то есть как множество байт
# почему - потому что.
print_request_info(
    requests.post(f"http://eu.httpbin.org/post", files={'file': open('test_file.txt', 'rb')})
)

200
{
  "args": {}, 
  "data": "", 
  "files": {
    "file": "Hello"
  }, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "154", 
    "Content-Type": "multipart/form-data; boundary=d08c2e480db31e848d78b4f0c4291251", 
    "Host": "eu.httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-63509085-3598cb635cab0b5b54edb1b5"
  }, 
  "json": null, 
  "origin": "178.70.82.198", 
  "url": "http://eu.httpbin.org/post"
}



Про разные виды Content Type можно прочитать, например, тут https://www.geeksforgeeks.org/http-headers-content-type/
Но, скорее всего, эта информация вам не пригодится.